In [1]:
kaggle=False
kaggle_path='/kaggle/input/nlp-getting-started/train.csv'
local_path='data/train.csv'
import torch

%load_ext autoreload
%autoreload 2

if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

if kaggle==False:
    device = torch.device("cpu")

from torch.utils.data import DataLoader
import pandas as pd
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from dataset import *
from transformer_arch import *
from learner import *


In [2]:

df=pd.read_csv(kaggle_path if kaggle else local_path)
df.head(5)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [3]:
%run preprocess.ipynb

preprocess_results=preprocess(df)
text_pipeline=preprocess_results['text_pipeline']
vocab_size=preprocess_results['vocab_size']
train_df=preprocess_results['train_df']
test_df=preprocess_results['test_df']
word_to_idx=preprocess_results['word_to_idx']
idx_to_word=preprocess_results['idx_to_word']
special_symbols=preprocess_results['special_symbols']


In [4]:
text_pipeline("This is a test sentence", word_to_idx)

[1639, 101, 27, 9219, 0]

In [5]:
disaster_tweets_dataset = DisasterTweetsDataset(df, vocab_size=vocab_size, text_pipeline=text_pipeline, word_to_idx=word_to_idx)
x=disaster_tweets_dataset[0]
print(x)


tensor([  74,    0,  869,   47, 7909,    7,  113,   16, 2907, 2093,    0,    0,
         230,  346,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1])


In [6]:


# test all together
df_to_test=df.sample(10)
dataset_to_test = DisasterTweetsDataset(df_to_test, vocab_size=vocab_size, sequence_length=30, text_pipeline=text_pipeline, word_to_idx=word_to_idx)
dataloader_to_test = DataLoader(dataset_to_test, batch_size=4, shuffle=True)
model = Seq2SeqTransformer(
    encoder=TransformerEncoder(vocab_size=vocab_size, emb_size=512, nhead=8, num_layers=6, dim_feedforward=2048, dropout=0.1),
    decoder=TransformerDecoder(vocab_size=vocab_size, emb_size=512, nhead=8, num_layers=6, dim_feedforward=2048, dropout=0.1),
    vocab_size=vocab_size,
    device=device,
    special_symbols=special_symbols
)

model.to(device)


# Beispielaufruf der Funktion

    
for x in dataloader_to_test:  
    print("x.shape", x.shape)

    # check if x contains values > vocab_size or < 0
    assert (x >= 0).all() and (x < vocab_size).all() , "x contains values outside of vocab_size"
    assert (x == x.long()).all(), "x is not of type long"
    assert (x.shape[1] == 30), "sequence length is not 2"
    assert (torch.isnan(x).sum().item() == 0), "x contains nan"
    output, y = model(x)

    print("model output shape: " , output.shape)
    print("nan count in output: ", torch.isnan(output).sum().item())
    
    # map output to vocab for each batch
    
    # output is batch_size, sequence_length, vocab_size
    # apply idx_to_word for each batch
    output_text = get_text_from_model_output(output, idx_to_word)
    print("model output: " , output_text)
    # Modify the line where the input tensor is created


x.shape torch.Size([4, 30])
nan count in embedding:  0
Max value in pos encoding: 2.368082046508789, Min value pos encoding: -2.2676241397857666
stddev value pos encoding: 0.8655033707618713, mean value pos encoding: 0.5251463055610657
nan count in pos encoder:  0
nan count in src:  0
nan count in src_mask:  0
nan count in src_padding_mask:  0
nan count in encoder:  10240
model output shape:  torch.Size([4, 30, 10000])
nan count in output:  200000
model output:  Adult tlk post compensate woes woes latest ElectronicMusic woes Dry ready Dry Hiroshima.70000 @termn8r13 ready pajamas tlk tlk tlk woes YA woes NO stlnd tlk <unk> <unk> <unk> <unk> <unk>
x.shape torch.Size([4, 30])
nan count in embedding:  0
Max value in pos encoding: 2.368126630783081, Min value pos encoding: -2.27201771736145
stddev value pos encoding: 0.8650890588760376, mean value pos encoding: 0.5176934003829956
nan count in pos encoder:  0
nan count in src:  0
nan count in src_mask:  0
nan count in src_padding_mask:  0
na

/opt/homebrew/Caskroom/miniconda/base/envs/torch_ds/lib/python3.10/site-packages/torch/nn/functional.py:5038: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(


In [7]:

# Hyperparameters
LR = 0.001  # learning rate
BATCH_SIZE = 128 # batch size for training


model = Seq2SeqTransformer(
    encoder=TransformerEncoder(vocab_size=vocab_size, emb_size=128, nhead=2, num_layers=2, dim_feedforward=512, dropout=0.1),
    decoder=TransformerDecoder(vocab_size=vocab_size, emb_size=128, nhead=2, num_layers=2, dim_feedforward=512, dropout=0.1),
    vocab_size=vocab_size,
    device=device,
    special_symbols=special_symbols
)

train_dataset=DisasterTweetsDataset(train_df, vocab_size=vocab_size, text_pipeline=text_pipeline, word_to_idx=word_to_idx)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True)
valid_dataset=DisasterTweetsDataset(test_df, vocab_size=vocab_size, text_pipeline=text_pipeline, word_to_idx=word_to_idx)
valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE)

loss_func = torch.nn.CrossEntropyLoss()
#
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10.0, gamma=0.9)   # every 3 epochs, LR is multiplied by 0.7

learner=Learner(model, train_dataloader, valid_dataloader, optimizer, scheduler, loss_func, device)

In [8]:

learner.fit_epochs(10)


nan count in embedding:  0
Max value in pos encoding: 1.7395097017288208, Min value pos encoding: -1.736190676689148
stddev value pos encoding: 0.7340680956840515, mean value pos encoding: 0.31255418062210083
nan count in pos encoder:  0
nan count in src:  0
nan count in src_mask:  0
nan count in src_padding_mask:  0
nan count in encoder:  1097728
nan values in output tensor(113920000)
loss of batch tensor(nan, grad_fn=<NllLoss2DBackward0>)


In [ ]:
learner.fit_epochs(10)


nan count in embedding:  1638400
Max value in pos encoding: nan, Min value pos encoding: nan
stddev value pos encoding: nan, mean value pos encoding: nan
nan count in pos encoder:  1638400
nan count in src:  0
nan count in src_mask:  0
nan count in src_padding_mask:  0
nan count in encoder:  1638400
nan values in output tensor(128000000)
loss of batch tensor(nan, grad_fn=<NllLoss2DBackward0>)
nan count in embedding:  1638400
Max value in pos encoding: nan, Min value pos encoding: nan
stddev value pos encoding: nan, mean value pos encoding: nan
nan count in pos encoder:  1638400
nan count in src:  0
nan count in src_mask:  0
nan count in src_padding_mask:  0
nan count in encoder:  1638400
nan values in output tensor(128000000)
loss of batch tensor(nan, grad_fn=<NllLoss2DBackward0>)
nan count in embedding:  1638400
Max value in pos encoding: nan, Min value pos encoding: nan
stddev value pos encoding: nan, mean value pos encoding: nan
nan count in pos encoder:  1638400
nan count in src:  

In [ ]:
learner.fit_epochs(10)


-----------------------------------------------------------
| end of epoch   0 | loss:   nans 
-----------------------------------------------------------
| end of epoch   1 | loss:   nans 
-----------------------------------------------------------
| end of epoch   2 | loss:   nans 


KeyboardInterrupt: 

In [ ]:
local_test_path='test.csv'
kaggle_test_path='/kaggle/input/nlp-getting-started/test.csv'

df_submission_test_data=pd.read_csv(kaggle_test_path if kaggle else local_test_path)
df_submission_test_data.sample(5)

,id,keyword,location,text
2191,7336,nuclear%20reactor,Helsinki,Finnish Nuclear Plant to Move Ahead After Fina...
3102,10267,war%20zone,MDS af ?,Saipan looks like a war zone though
2008,6761,lightning,NaN,Baby you're like lightning in a bottle
2111,7079,meltdown,"St. Catherine, Jamaica, W.I.",CommoditiesåÊAre Crashing Like It's 2008 All O...
622,2029,casualties,Ngayogyakarta Hadiningrat,Memorial day of 70 years Hiroshima and Nagasak...


In [ ]:
# label encode keyword using trained label encoder
df_submission_test_data['keyword'] = le.transform(df_submission_test_data['keyword'].fillna(''))

In [ ]:
def collate_batch_test_data(batch):
    x, keywords = zip(*batch)
    # collate for embedding bag
    empty_dummy_for_fixed_length = torch.zeros(sequence_length, dtype=torch.long)
    x = torch.stack([torch.cat((text, empty_dummy_for_fixed_length))[:sequence_length] for text in x])
    keywords=torch.stack(keywords)
    return x, keywords

In [ ]:
model.eval()
preds=[]
test_dataset=DisasterTweetsDataset(df_submission_test_data, vocab_size=vocab_size, test=True)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch_test_data)
with torch.no_grad():
    for idx, (x, keywords) in enumerate(test_dataloader):
        predicted_label = model(x, keywords)
        preds.extend(predicted_label.argmax(1).tolist())

df_submission_test_data["target"]=preds
df_submission_test_data[["id", "target"]].to_csv("submission_04.csv", index=False)

In [ ]:
# Score: 0.75237